# **Examples for CDC '25**

## **Code Example**

### **Imports**

In [ ]:
from pyspect import *
from pyspect.set_builder import *
from pyspect.primitives.ltl import *

### **Hyperparameters**

In [ ]:
AXIS_NAMES = ['x', 'v'] # [m, m/s]
MAX_BOUNDS = [750,  30] # 500m, 30 mps ~= 110 kmph
MIN_BOUNDS = [  0,   0] #   0m,  0 mps
GRID_SHAPE = ( 91,  91)

MAX_ACCEL = 1.0     # [mps2]
TIME_STEP = 0.5     # [s]
TIME_HORIZON = 40   # [s]

### **Program**

#### Task definition

1. Define the different regions. These can be are "constants" in the task specification.
2. Write the specification $\varphi = (p_\text{hw} \lor p_\text{r}) \:\mathsf{U}\: \verb|home|$, where $p_\text{hw}, p_\text{r}, \verb|home| \in \mathsf{AP}$. While $p_\text{hw} \leftrightarrow z \in \verb|HIGHWAY|$ and $p_\text{r} \leftrightarrow z \in \verb|RESIDENTIAL|$, the proposition $\verb|home|$ is implicit since we have not yet defined where "home" is. For the explicit propositions $p_\text{hw}$ and $p_\text{r}$, we simply use their corresponding sets in the specification in their stead.

In [ ]:
# (1) Define the different regions.

HIGHWAY     = Union(BoundedSet(x=(..., 400), v=(15, ...)),   # highway
                    BoundedSet(x=(300, 420), v=(10,  20)))   # offramp
RESIDENTIAL = BoundedSet(x=(400, ...), v=( 5, 20))           # residential

# (2) Write the specification.

CITY = OR(HIGHWAY, RESIDENTIAL) # Use sets in place of propositions
TASK = UNTIL(CITY, 'home')

#### Construct the TLT

In [ ]:
# Select the set of primitive TLTs 
# This, in de facto, determines the language
TLT.select(ContinuousLTL)

# Define the home region (implicit/variable in TASK).
H = BoundedSet(x=(700, 749), v=(1,  7), t=(TIME_HORIZON-2, ...))

# Create the TLT and set the proposition 'home'
objective = TLT(TASK).where(home=H)

#### Initialize the implementation object

In [ ]:
from pyspect.impls.hj_reachability import TVHJImpl
from pyspect.plotting.levelsets import *

import hj_reachability as hj
from hj_reachability.systems import *

dynamics = dict(cls=DoubleIntegrator,
                min_accel=-MAX_ACCEL,
                max_accel=+MAX_ACCEL)

impl = TVHJImpl(dynamics, 
                AXIS_NAMES,
                MIN_BOUNDS,
                MAX_BOUNDS,
                GRID_SHAPE,
                TIME_HORIZON,
                time_step=TIME_STEP)

## Run the reachability program

# `out` will have the same object type that `impl` operates with.
# For HJImpl(...), `out` will be a numpy arrays containing the gridded value function.
out = objective.realize(impl)

## Plot

plot3D_levelset(
    out,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    eye=EYE_MH_W,
)

## **Specification: $\square \psi$**

### **Imports**

In [ ]:
# Implementation independent

from time import time
from contextlib import contextmanager
from tqdm import tqdm

from pyspect import *
from pyspect.set_builder import *
from pyspect.primitives.ltl import *

# HJ specific

from pyspect.impls.hj_reachability import TVHJImpl
from pyspect.plotting.levelsets import *

import hj_reachability as hj
from hj_reachability.systems import DoubleIntegrator as HJDoubleIntegrator

# HZ specific

from pyspect.plotting.zonotopes import _hz2hj

from hz_reachability.hz_impl import TVHZImpl
from hz_reachability.systems.cars import *
from hz_reachability.systems.integrators import DoubleIntegrator as HZDoubleIntegrator
from hz_reachability.spaces import EmptySpace

In [ ]:
@contextmanager
def timectx(msgfunc):
    """Context manager to time a block of code."""
    start = time()
    yield
    end = time()
    print(msgfunc(end-start))

def print_hzinfo(hz):
    if isinstance(hz, list):
        nz, ng, nb, nc = \
            np.array([[_out.dim, _out.ng, _out.nb, _out.nc]
                    for _out in hz]).max(axis=0)
    else:
        nz,ng,nb,nc = hz.dim, hz.ng, hz.nb, hz.nc
    print(f"nz: {nz}, ng: {ng}, nb: {nb}, nc: {nc}")

### **Hyperparameters**

In [ ]:
AXIS_NAMES = ['x',  'v'] # [m, m/s]
MAX_BOUNDS = [+100, +20] # 500m, 30 mps ~= 110 kmph
MIN_BOUNDS = [-100, -20] #   0m,  0 mps
GRID_SHAPE = (  91,  91)

MAX_ACCEL = 1.0     # [mps2]
TIME_STEP = 0.5     # [s]
TIME_HORIZON = 40   # [s]

### **Program**

In [ ]:
## SPECIFICATION

T = BoundedSet(x=(-50,  +50))

phi = ALWAYS(T)

## CONSTRUCT TLT

TLT.select(ContinuousLTL)
tree = TLT(phi)

#### HJ Implementation

In [ ]:
dynamics = dict(cls=HJDoubleIntegrator,
                min_accel=-MAX_ACCEL,
                max_accel=+MAX_ACCEL)

impl = TVHJImpl(dynamics, 
                AXIS_NAMES,
                MIN_BOUNDS,
                MAX_BOUNDS,
                GRID_SHAPE,
                TIME_HORIZON,
                time_step=TIME_STEP)

with timectx(lambda t: f"Realization with HJ took {t:.2f} seconds"):
    out = tree.realize(impl)

## Plot

plot3D_levelset(
    out,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    colorscale='greens',
    eye=EYE_MH_W,
)

#### HZ Implementation

In [ ]:
space = EmptySpace(MIN_BOUNDS, MAX_BOUNDS)

dynamics = HZDoubleIntegrator(max_accel=MAX_ACCEL, dt=TIME_STEP)

impl = TVHZImpl(dynamics, space, AXIS_NAMES, time_horizon=TIME_HORIZON, time_step=TIME_STEP)
# impl.enable_reduce = True

with timectx(lambda t: f"Realization with HZ took {t:.2f} seconds"):
    out = tree.realize(impl)

print_hzinfo(out)

## Plot

if not isinstance(out, list):
    vf = np.array([_hz2hj(out, MIN_BOUNDS, MAX_BOUNDS, GRID_SHAPE)] * impl.N)
else:
    vf = np.array([_hz2hj(_out, MIN_BOUNDS, MAX_BOUNDS, GRID_SHAPE) for _out in tqdm(out)])

plot3D_levelset(
    vf,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    colorscale='blues',
    eye=EYE_MH_W,
)

## **Specification: $\square \lozenge \psi$**

### **Imports**

In [ ]:
# Implementation independent

from time import time
from contextlib import contextmanager
from tqdm import tqdm

from pyspect import *
from pyspect.set_builder import *
from pyspect.primitives.ltl import *

# HJ specific

from pyspect.impls.hj_reachability import TVHJImpl
from pyspect.plotting.levelsets import *

import hj_reachability as hj
from hj_reachability.systems import DoubleIntegrator as HJDoubleIntegrator

# HZ specific

from pyspect.plotting.zonotopes import _hz2hj

from hz_reachability.hz_impl import TVHZImpl
from hz_reachability.systems.cars import *
from hz_reachability.systems.integrators import DoubleIntegrator as HZDoubleIntegrator
from hz_reachability.spaces import EmptySpace

In [ ]:
@contextmanager
def timectx(msgfunc):
    """Context manager to time a block of code."""
    start = time()
    yield
    end = time()
    print(msgfunc(end-start))

def print_hzinfo(hz):
    if isinstance(hz, list):
        nz, ng, nb, nc = \
            np.array([[_out.dim, _out.ng, _out.nb, _out.nc]
                    for _out in hz]).max(axis=0)
    else:
        nz,ng,nb,nc = hz.dim, hz.ng, hz.nb, hz.nc
    print(f"nz: {nz}, ng: {ng}, nb: {nb}, nc: {nc}")

### **Hyperparameters**

In [ ]:
AXIS_NAMES = ['x',  'v'] # [m, m/s]
MAX_BOUNDS = [+100, +20] # 500m, 30 mps ~= 110 kmph
MIN_BOUNDS = [-100, -20] #   0m,  0 mps
GRID_SHAPE = (  91,  91)

MAX_ACCEL = 1.0     # [mps2]
TIME_STEP = 0.5     # [s]
TIME_HORIZON = 40   # [s]

### **Program**

In [ ]:
## SPECIFICATION

T = BoundedSet(x=(-50,  +50))

phi = ALWAYS(EVENTUALLY(T))

## CONSTRUCT TLT

TLT.select(ContinuousLTL)
tree = TLT(phi)

#### HJ Implementation

In [ ]:
dynamics = dict(cls=HJDoubleIntegrator,
                min_accel=-MAX_ACCEL,
                max_accel=+MAX_ACCEL)

impl = TVHJImpl(dynamics, 
                AXIS_NAMES,
                MIN_BOUNDS,
                MAX_BOUNDS,
                GRID_SHAPE,
                TIME_HORIZON,
                time_step=TIME_STEP)

with timectx(lambda t: f"Realization with HJ took {t:.2f} seconds"):
    out = tree.realize(impl)

## Plot

plot3D_levelset(
    out,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    colorscale='greens',
    eye=EYE_MH_W,
)

#### HZ Implementation

In [ ]:
space = EmptySpace(MIN_BOUNDS, MAX_BOUNDS)

dynamics = HZDoubleIntegrator(max_accel=MAX_ACCEL, dt=TIME_STEP)

impl = TVHZImpl(dynamics, space, AXIS_NAMES, time_horizon=TIME_HORIZON, time_step=TIME_STEP)
# impl.enable_reduce = True

with timectx(lambda t: f"Realization with HZ took {t:.2f} seconds"):
    out = tree.realize(impl)

print_hzinfo(out)

## Plot

if not isinstance(out, list):
    vf = np.array([_hz2hj(out, MIN_BOUNDS, MAX_BOUNDS, GRID_SHAPE)] * impl.N)
else:
    vf = np.array([_hz2hj(_out, MIN_BOUNDS, MAX_BOUNDS, GRID_SHAPE) for _out in tqdm(out)])

plot3D_levelset(
    vf,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    colorscale='blues',
    eye=EYE_MH_W,
)